# SPR 2026 - mDeBERTa V3

Fine-tuning do mDeBERTa V3 para classificação BI-RADS.

**Modelo:** microsoft/mdeberta-v3-base

**Formato:** Code Competition (Kaggle) / Google Colab

In [ ]:
# ============================================================
# SETUP - Ambiente, Download de Dados e Google Drive
# ============================================================
import os
import sys

IS_KAGGLE = os.path.exists('/kaggle/input')
IS_COLAB = 'google.colab' in sys.modules

print(f"Ambiente: {'Kaggle' if IS_KAGGLE else 'Colab' if IS_COLAB else 'Local'}")

if IS_KAGGLE:
    DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
    OUTPUT_DIR = '/kaggle/working'
elif IS_COLAB:
    # Montar Google Drive para salvar outputs
    from google.colab import drive
    drive.mount('/content/drive')
    
    DRIVE_OUTPUT = '/content/drive/MyDrive/SPR_2026_outputs'
    os.makedirs(DRIVE_OUTPUT, exist_ok=True)
    OUTPUT_DIR = DRIVE_OUTPUT
    
    # Download dados Kaggle
    !pip install kaggle transformers accelerate sentencepiece -q
    from google.colab import userdata
    os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
    os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
    !mkdir -p ~/.kaggle
    !echo '{"username":"'$KAGGLE_USERNAME'","key":"'$KAGGLE_KEY'"}' > ~/.kaggle/kaggle.json
    !chmod 600 ~/.kaggle/kaggle.json
    !kaggle competitions download -c spr-2026-mammography-report-classification -q
    !mkdir -p data && unzip -o -q spr-2026-mammography-report-classification.zip -d data/
    DATA_DIR = 'data'
else:
    DATA_DIR = '../data'
    OUTPUT_DIR = '../submissions'
    os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"DATA_DIR: {DATA_DIR}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from transformers import (
    AutoModelForSequenceClassification, AutoTokenizer,
    Trainer, TrainingArguments, EarlyStoppingCallback
)
import warnings
warnings.filterwarnings('ignore')

# Config
SEED = 42
MODEL_NAME = 'microsoft/mdeberta-v3-base'
MAX_LENGTH = 512
BATCH_SIZE = 8
EPOCHS = 5
LR = 1e-5
PATIENCE = 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

np.random.seed(SEED)
torch.manual_seed(SEED)

## 1. Carregar Dados

In [ ]:
train_df = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
print(f"Train: {train_df.shape}")

test_path = os.path.join(DATA_DIR, 'test.csv')
if os.path.exists(test_path):
    test_df = pd.read_csv(test_path)
    print(f"Test: {test_df.shape}")
else:
    test_df = None
    print("test.csv não disponível - será carregado no runtime Kaggle")

## 2. Dataset

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        enc = self.tokenizer(
            str(self.texts[idx]),
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
        )
        item = {
            'input_ids': enc['input_ids'].squeeze(),
            'attention_mask': enc['attention_mask'].squeeze(),
        }
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

## 3. Treinar com Early Stopping

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['report'].tolist(),
    train_df['target'].tolist(),
    test_size=0.1,
    stratify=train_df['target'],
    random_state=SEED
)

train_ds = TextDataset(train_texts, train_labels, tokenizer, MAX_LENGTH)
val_ds = TextDataset(val_texts, val_labels, tokenizer, MAX_LENGTH)

print(f"Train: {len(train_ds)}, Val: {len(val_ds)}")

In [ ]:
def compute_metrics(eval_pred):
    preds = np.argmax(eval_pred.predictions, axis=1)
    return {'f1_macro': f1_score(eval_pred.label_ids, preds, average='macro')}

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=7)

args = TrainingArguments(
    output_dir=f'{OUTPUT_DIR}/deberta',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE*2,
    learning_rate=LR,
    weight_decay=0.01,
    warmup_ratio=0.1,
    fp16=torch.cuda.is_available(),
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1_macro',
    greater_is_better=True,
    report_to='none',
    seed=SEED,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)],
)

trainer.train()

In [ ]:
val_preds = trainer.predict(val_ds)
val_f1 = f1_score(val_labels, np.argmax(val_preds.predictions, axis=1), average='macro')
print(f"Validation F1-Macro: {val_f1:.4f}")

## 4. Gerar Submissão

In [ ]:
if test_df is not None:
    test_ds = TextDataset(test_df['report'].tolist(), None, tokenizer, MAX_LENGTH)
    test_preds = trainer.predict(test_ds)
    predictions = np.argmax(test_preds.predictions, axis=1)
    
    submission = pd.DataFrame({'ID': test_df['ID'], 'target': predictions})
    submission.to_csv(os.path.join(OUTPUT_DIR, 'submission.csv'), index=False)
    print(f"Submissão salva!")
    print(submission['target'].value_counts().sort_index())
else:
    print("Test não disponível - submeta no Kaggle")

In [ ]:
if IS_COLAB and test_df is not None:
    from google.colab import files
    files.download('submission.csv')